In [1]:
import pandas as pd
import numpy as np
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display


class DataLoader:
    def __init__(self, cuentas_map_filepath):
        self.cuentas_map = self.load_cuentas_map(cuentas_map_filepath)

    def load_cuentas_map(self, filepath):
        cuentas_rh = pd.read_excel(filepath)
        cuentas_rh["Cuenta"] = cuentas_rh["Cuenta"].astype(str)
        cuentas_rh["Cuenta Conc"] = cuentas_rh["Cuenta Conc"].apply(lambda x: str(x) if pd.notnull(x) else "")
        
        cuentas_map = {}
        
        for _, row in cuentas_rh.iterrows():
            cuenta = row["Cuenta"]
            concepto = row["Concepto"]
            clasificacion = row["Clasificacion"]
            cuenta_conc = row["Cuenta Conc"]
            tipo_cuenta = row["Tipo de Cuenta"]
            
            if cuenta not in cuentas_map:
                cuentas_map[cuenta] = {"Concepto": concepto, "Clasificacion": clasificacion, "Cuenta Conc": [], "Tipo de Cuenta": tipo_cuenta}
                
            if cuenta_conc:
                for conc in cuenta_conc.split(","):
                    conc = conc.strip()
                    if conc:
                        cuentas_map[cuenta]["Cuenta Conc"].append(conc)
        
        return cuentas_map

    def load_data_from_folder(self, folder_path, columns):
        files = [f for f in os.listdir(folder_path) if f.endswith("txt")]
        dataframes = []

        for file in files:
            file_path = os.path.join(folder_path, file)
            data = pd.read_csv(file_path, encoding="latin1", delimiter="\t")
            data = data.drop(data.columns[0], axis=1)
            data.columns = columns
            data = data.fillna("")
            dataframes.append(data)

        df = pd.concat(dataframes, ignore_index=True)
        
        df["Cuenta"] = df["Cuenta"].astype(str)
        df["Concepto"] = df["Cuenta"].map(lambda x: self.cuentas_map[x]["Concepto"] if x in self.cuentas_map else "")
        
        return df

    def preprocess_data(self, df):
        df["Cuenta"] = df["Cuenta"].astype(str)
        df["Importe"] = df["Importe"].str.strip()
        df["Importe"] = df["Importe"].str.replace(",", "").astype(float)
        df["Fecha de documento"] = pd.to_datetime(df["Fecha de documento"], dayfirst=True)
        df["Fe.contabilización"] = pd.to_datetime(df["Fe.contabilización"], dayfirst=True)
        df["Año Documento"] = df['Fecha de documento'].dt.year
        df["Mes"] = df['Fe.contabilización'].dt.month
        df["Año"] = df["Fe.contabilización"].dt.year
        
        df["Concepto"] = df["Cuenta"].map(lambda x: self.cuentas_map[x]["Concepto"] if x in self.cuentas_map else "")
        
        # Reorder columns
        col = df.columns.tolist()
        if "Concepto" in col:
            col.remove("Concepto")
        col.remove("Año")
        col.remove("Mes")
        index_cuenta = col.index("Cuenta")
        if "Concepto" in df.columns:
            col.insert(index_cuenta + 1, "Concepto")
        col.insert(index_cuenta + 1, "Año")
        col.insert(index_cuenta + 2, "Mes")
        
        return df[col]

    def calculate_centro_de_beneficio(self, row):
        sociedad = row['Sociedad']
        centro_beneficio = row['Centro de beneficio']

        if sociedad == 1:
            return centro_beneficio
        else:
            sociedad_str = str(sociedad)
            centro_beneficio_str = str(centro_beneficio)
            prefix = sociedad_str + "0"

            if centro_beneficio_str.startswith(prefix):
                return int(centro_beneficio_str[len(prefix):])
            else:
                return centro_beneficio

In [3]:
class BudgetHandler:
    def __init__(self, budget_filepaths):
        """
        budget_filepaths should be a dictionary where the keys are years and the values are the file paths.
        Example: {2022: "path_to_2022_budget_file.xlsx", 2023: "path_to_2023_budget_file.xlsx"}
        """
        self.budget_filepaths = budget_filepaths

    def load_budget_data(self):
        presupuesto_todo = {}
        sociedad_dicts = {}

        for year, filepath in self.budget_filepaths.items():
            ptto_path = pd.ExcelFile(filepath)
            sociedad_dict = ptto_path.parse("Sociedades").set_index("Nombre")["Sociedad"].to_dict()
            sociedad_dicts[year] = sociedad_dict

            for key in sociedad_dict.keys():
                if key in ptto_path.sheet_names:
                    df = ptto_path.parse(key)
                    new_header = df.iloc[0]
                    df = df[1:]
                    df.columns = new_header
                    df.rename(columns={"CONCEP COMPA": "Concepto", "CUENTA": "Cuenta"}, inplace=True)
                    df["Concepto"] = df["Concepto"].astype(str).str.extract(r"(\D+)", expand=False).str.strip()
                    df["Cuenta"] = df["Cuenta"].astype(str)
                    df = df[df["Concepto"] != "TOTAL"]
                    df["Año"] = year
                    presupuesto_todo.setdefault(year, {})[key] = df
        return presupuesto_todo, sociedad_dicts

    def melt_and_preprocess_budget(self, presupuesto_todo):
        months = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
                  "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
        mes_map = {
            'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
            'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12}

        for year, presupuesto in presupuesto_todo.items():
            for key, df in presupuesto.items():
                df = df.melt(id_vars=["Concepto", "Cuenta", "Año"], value_vars=months, var_name="Mes", value_name="Provision")
                df["Mes"] = df["Mes"].map(mes_map)
                df['Provision'] = df['Provision'].astype(float)
                presupuesto_todo[year][key] = df
        return presupuesto_todo

    def filter_combine_budget(self, datos, presupuesto_todo, sociedad_dicts, cuentas_map):
        presupuesto_combined = {}

        cuentas_validas = datos["Cuenta"].astype(str).unique()

        for year, presupuesto in presupuesto_todo.items():
            for key, df in presupuesto.items():
                df_filtrado = df[df["Cuenta"].isin(cuentas_validas)]
                df_filtrado["Concepto"] = df_filtrado["Cuenta"].map(cuentas_map)

                # Exclude concepts with zero provision in all months for the year
                provision_cero = df_filtrado[df_filtrado['Provision'] == 0].groupby(['Concepto', 'Año']).size().reset_index(name='ContadorCero')
                conceptos_cero_total = provision_cero[provision_cero['ContadorCero'] == 12]["Concepto"].unique()
                df_filtrado = df_filtrado[~df_filtrado["Concepto"].isin(conceptos_cero_total)]

                # Sort the filtered DataFrame
                df_sorted = df_filtrado.sort_values(by=["Concepto", "Mes"])

                presupuesto_combined.setdefault(year, {})[key] = df_sorted

        # Convert combined dictionary to DataFrame and apply additional formatting
        all_presupuestos = []
        for year, presupuesto in presupuesto_combined.items():
            for key, df in presupuesto.items():
                df["Sociedad"] = sociedad_dicts[year].get(key, key)  # Map sociedad_dict to the DataFrame
                df.rename(columns={"Provision": "Importe"}, inplace=True)
                df["Cuenta"] = df["Cuenta"].astype(str)
                df["Concepto"] = df["Cuenta"].map(cuentas_map)
                df["Año"] = year
                all_presupuestos.append(df)

        final_presupuesto = pd.concat(all_presupuestos, ignore_index=True)

        return final_presupuesto

In [5]:
class DataPreparation:
    def __init__(self, data_loader, budget_handler):
        self.data_loader = data_loader
        self.budget_handler = budget_handler

    def classify_conceptos(self, datos, conc_250):
        conciliable_conceptos = set()
        no_conciliable_conceptos = set()

        for cuenta, info in self.data_loader.cuentas_map.items():
            concepto = info["Concepto"]
            cuenta_conc_list = info["Cuenta Conc"]
            if datos[datos["Concepto"] == concepto].empty:
                continue
            
            if any(cuenta_conc in conc_250["Cuenta"].astype(str).unique() for cuenta_conc in cuenta_conc_list):
                conciliable_conceptos.add(concepto)
            else:
                no_conciliable_conceptos.add(concepto)

        return list(conciliable_conceptos), list(no_conciliable_conceptos)

    def remove_rows_from_conc_250(self, conc_250, removed_conceptos):
        conc_cuentas_to_remove = set()

        for cuenta, info in self.data_loader.cuentas_map.items():
            if info["Concepto"] in removed_conceptos:
                conc_cuentas_to_remove.update(info["Cuenta Conc"])

        conc_250 = conc_250[~conc_250["Cuenta"].isin(conc_cuentas_to_remove)]
        return conc_250

    def main(self):
        columns = ["Cuenta", "Fe.contabilización", "Fecha de documento", "Clase", "Nº documento", "Importe", "Moneda", "Clave contabiliz", "Centro de beneficio", "Sociedad", "Referencia a factura", "Cta.contrapartida", "Texto", "Usuario"]

        datos = self.data_loader.load_data_from_folder(r"C:\Users\A8064444\Documents\2024\11 2024\FINAL SERVICIO MEDICO\Datos", columns)
        datos = self.data_loader.preprocess_data(datos)
        datos['Calculated_Centro_de_beneficio'] = datos.apply(self.data_loader.calculate_centro_de_beneficio, axis=1)
        
        conc_250 = self.data_loader.load_data_from_folder(r"C:\Users\A8064444\Documents\2024\11 2024\FINAL SERVICIO MEDICO\Datos Conc", columns)
        conc_250 = self.data_loader.preprocess_data(conc_250)
        conc_250['Calculated_Centro_de_beneficio'] = conc_250.apply(self.data_loader.calculate_centro_de_beneficio, axis=1)
        
        conciliable_conceptos, no_conciliable_conceptos = self.classify_conceptos(datos, conc_250)

        removed_conceptos = set()
        removed_conceptos.update(datos.loc[datos["Concepto"] == "UNIFORMES", "Concepto"].unique())
        removed_conceptos.update(datos.loc[datos["Concepto"] == "CENA DE FIN DE AÑO", "Concepto"].unique())
        removed_conceptos.update(datos.loc[datos["Concepto"] == "HONORARIOS ADMINISTRADOS X RH", "Concepto"].unique())

        datos = datos.loc[~(datos["Concepto"] == "UNIFORMES")]
        datos = datos.loc[~(datos["Concepto"] == "CENA DE FIN DE AÑO")]
        datos = datos.loc[~(datos["Concepto"] == "HONORARIOS ADMINISTRADOS X RH")]

        conc_250 = self.remove_rows_from_conc_250(conc_250, removed_conceptos)

        return datos, conc_250, conciliable_conceptos, no_conciliable_conceptos

    def execute_budget_analysis(self, datos):
        budget_files = {
            2023: r"C:\Users\A8064444\Documents\2024\11 2024\FINAL SERVICIO MEDICO\PTTO\12 PPTO 2023 DICIEMBRE.xlsx",
            2024: r"C:\Users\A8064444\Documents\2024\11 2024\FINAL SERVICIO MEDICO\PTTO\10 PPTO 2024 OCTUBRE.xlsx"
        }

        self.budget_handler = BudgetHandler(budget_files)
        presupuesto_todo, sociedad_dicts = self.budget_handler.load_budget_data()
        presupuesto_todo = self.budget_handler.melt_and_preprocess_budget(presupuesto_todo)
        cuentas_map = datos.set_index("Cuenta")["Concepto"].to_dict()
        presupuesto_combined = self.budget_handler.filter_combine_budget(datos, presupuesto_todo, sociedad_dicts, cuentas_map)

        return presupuesto_combined

# Instantiate and execute the process
if __name__ == "__main__":
    data_loader = DataLoader(r"C:\Users\A8064444\Documents\2024\09 2024\IMSS, CESANTIA etc\Cuentas Map\Conc Cuentas 2.xlsx")
    budget_files = {
        2023: r"C:\Users\A8064444\Documents\2024\11 2024\FINAL SERVICIO MEDICO\PTTO\12 PPTO 2023 DICIEMBRE.xlsx",
        2024:  r"C:\Users\A8064444\Documents\2024\11 2024\FINAL SERVICIO MEDICO\PTTO\10 PPTO 2024 OCTUBRE.xlsx"
    }
    budget_handler = BudgetHandler(budget_files)
    data_preparation = DataPreparation(data_loader, budget_handler)
    
    datos, conc_250, conciliable_conceptos, no_conciliable_conceptos = data_preparation.main()
    presupuesto_combined = data_preparation.execute_budget_analysis(datos)
    presupuesto_combined.rename(columns={"Importe":"Presupuesto"},inplace=True)
    
    
    pd.options.display.float_format = "${:,.2f}".format
    print(datos.head())
    print(conc_250.head())
    print(presupuesto_combined.head())

       Cuenta   Año  Mes         Concepto Fe.contabilización  \
0  5106030100  2024    7  BOTIQUIN MEDICO         2024-07-12   
1  5106030100  2024    8  BOTIQUIN MEDICO         2024-08-02   
2  5106030100  2024    3  BOTIQUIN MEDICO         2024-03-08   
3  5106030100  2024    9  BOTIQUIN MEDICO         2024-09-23   
4  5106030100  2024   10  BOTIQUIN MEDICO         2024-10-21   

  Fecha de documento Clase  Nº documento  Importe Moneda  Clave contabiliz  \
0         2024-06-24    RE    5100028366   $44.00    MXN                81   
1         2024-07-04    RE    5100031887   $58.00    MXN                81   
2         2024-02-09    RE    5100008069  $980.61    MXN                81   
3         2024-08-15    RE    5100040627   $10.40    MXN                81   
4         2024-09-10    RE    5100045847   $29.72    MXN                81   

   Centro de beneficio  Sociedad  Referencia a factura  Cta.contrapartida  \
0                51003         1            5105736946            109

C:\Users\A8064444\AppData\Local\Temp\ipykernel_13264\2923225123.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["Concepto"] = df_filtrado["Cuenta"].map(cuentas_map)
C:\Users\A8064444\AppData\Local\Temp\ipykernel_13264\2923225123.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["Concepto"] = df_filtrado["Cuenta"].map(cuentas_map)
C:\Users\A8064444\AppData\Local\Temp\ipykernel_13264\2923225123.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [6]:
class FilterHelper:
    def __init__(self, datos, conc_250, cuentas_map):
        self.datos = datos
        self.conc_250 = conc_250
        self.cuentas_map = cuentas_map

    def get_filtered_datos(self, concepto, año, mes, sociedad):
        """
        Filter datos by concepto, año, mes, and sociedad.
        """
        self.datos["Index"] = self.datos.index
        return self.datos[(self.datos["Concepto"] == concepto) &
                          (self.datos["Año"] == año) &
                          (self.datos["Mes"] == mes) &
                          (self.datos["Sociedad"] == sociedad)]

    def get_filtered_conc(self, cuenta, año, mes, sociedad):
        """
        Filter conc_250 by cuenta, año, mes, and sociedad.
        """
        self.conc_250["Index"] = self.conc_250.index
        return self.conc_250[(self.conc_250["Cuenta"] == cuenta) &
                             (self.conc_250["Año"] == año) &
                             (self.conc_250["Mes"] == mes) &
                             (self.conc_250["Sociedad"] == sociedad)]

    def get_datos_by_day(self, filtered_datos):
        """
        Group filtered_datos by day and sum the "Importe".
        """
        return filtered_datos.groupby("Fe.contabilización").agg({
            "Importe": "sum",
            "Index": lambda x: x.index.tolist()
        })

    def get_datos_by_documento(self, filtered_datos):
        """
        Group filtered_datos by "Nº documento" and sum the "Importe".
        """
        return filtered_datos.groupby("Nº documento").agg({
            "Importe": "sum",
            "Index": lambda x: x.index.tolist()
        })

    def get_datos_by_referencia(self, filtered_datos):
        """
        Group filtered_datos by "Referencia a factura" and sum the "Importe".
        """
        return filtered_datos.groupby("Referencia a factura").agg({
            "Importe": "sum",
            "Index": lambda x: x.index.tolist()
        })

    def filter_by_account_type_1(self, filtered_datos):
        """
        Filter datos by 'Tipo de Cuenta == 1' (expenditures).
        """
        cuentas_type_1 = [
            cuenta for cuenta in filtered_datos["Cuenta"].unique()
            if cuenta in self.cuentas_map and self.cuentas_map[cuenta]["Tipo de Cuenta"] == 1
        ]
        return filtered_datos[filtered_datos["Cuenta"].isin(cuentas_type_1)]
    

    def filter_by_account_type_2(self, filtered_datos):
        """
        Filter datos by 'Tipo de Cuenta == 2' (provisions).
        """
        accounts_in_filtered_datos = filtered_datos["Cuenta"].unique()
        cuentas_type_2 = [
            cuenta for cuenta in accounts_in_filtered_datos
            if cuenta in self.cuentas_map and self.cuentas_map[cuenta]["Tipo de Cuenta"] == 2
        ]
        filtered_by_type_2 = filtered_datos[filtered_datos["Cuenta"].isin(cuentas_type_2)]
        return filtered_by_type_2

    def get_account_type(self, cuenta):
        """
        Classify whether an account is 'type_1' or 'type_2'.
        Uses the helper's cuentas_map to retrieve the account type.

        Args:
        - cuenta: The account (Cuenta) to classify.
        
        Returns:
        - int: 1 for 'type_1' account, 2 for 'type_2' account, or None if no type found.
        """
        cuenta_info = self.cuentas_map.get(cuenta)
        if cuenta_info:
            return cuenta_info.get("Tipo de Cuenta")  # Return the type of the account (1 or 2)
        return None  # Return None if no type is found
    
    def get_diferencia_provision_df(self, indices, concepto, año, mes, sociedad):
        """
        Separate diferencia_provision_df and gasto_real_df.
        """
        diferencia_provision_df = self.datos[(self.datos["Concepto"] == concepto) &
                                             (self.datos.index.isin(indices))]
        gasto_real_df = self.datos[(self.datos["Concepto"] == concepto) &
                                   (self.datos["Mes"]==mes)&
                                   (self.datos["Sociedad"]==sociedad)&
                                   (self.datos["Año"]==año)&
                                   (~self.datos.index.isin(indices))]
        
        return diferencia_provision_df, gasto_real_df

In [9]:
class BaseCalculator:
    def __init__(self, helper, tolerance=0.5):
        self.helper = helper
        self.tolerance = tolerance
        self.diferencias = {}  # Dictionary to store differences

    def check_tolerance(self, value1, value2, operation="subtract"):
        """
        Check if the absolute difference between value1 and value2 is within the tolerance.
        """
        if operation == "subtract":
            difference = abs(value1 - value2)
        elif operation == "add":
            difference = abs(value1 + value2)
        else:
            raise ValueError("Unsupported operation")
        
        return difference <= self.tolerance

    def get_diferencias(self, expected_value, comparison_sum, año, concepto, mes, sociedad, indices):
        """
        Calculate and store the difference between expected_value and comparison_sum if they don't match.
        """
        # Filter `datos` based on provided indices
        filtered_data = self.helper.datos.loc[indices]
        
        # Separate sums for type_1 and type_2 accounts
        type_1_account_sum = filtered_data[filtered_data["Cuenta"].apply(self.helper.get_account_type) == 1]["Importe"].sum()
        type_2_account_sum = filtered_data[filtered_data["Cuenta"].apply(self.helper.get_account_type) == 2]["Importe"].sum()
        
        # Calculate the adjusted comparison sum for mixed accounts
        if abs(type_1_account_sum) >= self.tolerance:
            adjusted_comparison_sum = type_1_account_sum - type_2_account_sum
        else:
            adjusted_comparison_sum = type_2_account_sum
            
        # Calculate the difference with the correct sign adjustment
        if expected_value != adjusted_comparison_sum:
            if adjusted_comparison_sum < 0:
                # Add adjusted_comparison_sum if it’s negative
                diferencia = expected_value + adjusted_comparison_sum
            else:
                # Subtract adjusted_comparison_sum if it’s positive
                diferencia = expected_value - adjusted_comparison_sum
    
            # Store the difference if it's above tolerance
            key = (año, concepto, mes, sociedad)
            self.diferencias[key] = diferencia
        else:
            print(f"No hay diferencia!")
    
    def prompt_user_for_indices(self, filtered_provision_datos, presupuesto, año, concepto, mes, sociedad):
        """
        Display available rows as a DataFrame and prompt the user to select indices for provision matching.
        """
        print(f"Presupuesto for Año: {año}, Concepto: {concepto}, Mes: {mes}, Sociedad: {sociedad}: ${presupuesto}")
        display(filtered_provision_datos)

        selected_indices = []
        while True:
            try:
                user_input = input("Porfavor, agregar los indices de la Provision separado por commas : ")
                selected_indices = [int(index.strip()) for index in user_input.split(",")]
                # Ensure the selected indices are valid
                if all(idx in filtered_provision_datos.index for idx in selected_indices):
                    break
                else:
                    print("Indice Invalido. Porfavor agregar indices válidos.")
            except ValueError:
                print("Porfavor, agregar los indices de la Provision separado por commas.")
        
        # Calculate total of selected provisions
        provision_sum = filtered_provision_datos.loc[selected_indices, "Importe"].sum()
        # Call get_diferencias to check and store any difference
        self.get_diferencias(presupuesto, provision_sum, año, concepto, mes, sociedad,selected_indices)

        return {"Indices": selected_indices, "Presupuesto": presupuesto}

In [11]:
class ProvisionCalculator(BaseCalculator):
    def __init__(self, helper, presupuesto_combined, tolerance=0.5):
        super().__init__(helper, tolerance)
        self.presupuesto_combined = presupuesto_combined
        self.cache = {}  # Cache para almacenar resultados de provisión

    def get_presupuesto_importe(self, año, concepto, mes, sociedad):
        """
        Obtener el importe total del presupuesto para los datos filtrados.
        """
        presupuesto_importe = self.presupuesto_combined[
            (self.presupuesto_combined["Año"] == año) &
            (self.presupuesto_combined["Concepto"] == concepto) &
            (self.presupuesto_combined["Mes"] == mes) &
            (self.presupuesto_combined["Sociedad"] == sociedad)
        ]["Presupuesto"].sum()
        
        return presupuesto_importe

    def get_cached_provision(self, concepto, año, mes, sociedad):
        """
        Devuelve la provisión desde la caché si ya existe, de lo contrario, calcula y la almacena en la caché.
        """
        cache_key = (año, concepto, mes, sociedad)

        # Verificar si el resultado ya está en la caché
        if cache_key in self.cache:
            print(f"Obteniendo resultado de provisión en caché para: {cache_key}")
            return self.cache[cache_key]

        # Si no está en la caché, calcular la provisión y almacenar el resultado
        provision_result = self.calculate_provision(concepto, año, mes, sociedad)
        self.cache[cache_key] = provision_result
        return provision_result

    def calculate_provision(self, concepto, año, mes, sociedad):
        """
        Calcular la provisión para un concepto, año, mes y sociedad específicos.
        """
        # Paso 1: Obtener el presupuesto correspondiente
        presupuesto = self.get_presupuesto_importe(año, concepto, mes, sociedad)
        filtered_datos = self.helper.get_filtered_datos(concepto, año, mes, sociedad)
        filtered_provision_datos = self.helper.filter_by_account_type_2(filtered_datos)

        # Paso 2: Intentar encontrar coincidencias exactas en la provisión individual
        match_count = 0
        match_index = []
        matched_gasto = 0

        for idx, row in filtered_provision_datos.iterrows():
            gasto_importe = row["Importe"]
            if self.check_tolerance(presupuesto, gasto_importe, operation="subtract"):
                match_count += 1
                match_index.append(idx)
                matched_gasto += gasto_importe

        if match_count == 1:
            # Devolver los índices si se encuentra una coincidencia exacta
            return {"Indices": match_index, "Presupuesto": presupuesto}

        # Paso 3: Agrupar por día y verificar coincidencias en el "Importe" agrupado
        aggregated_by_day = self.helper.get_datos_by_day(filtered_provision_datos)
        matched_day_indices = []
        
        for day, row in aggregated_by_day.iterrows():
            day_importe = row["Importe"]
            day_indices = row["Index"]
            
            if self.check_tolerance(presupuesto, day_importe, operation="subtract"):
                matched_day_indices.append(day_indices)

        # Si se encuentra exactamente un día coincidente
        if len(matched_day_indices) == 1:
            return {"Indices": matched_day_indices[0], "Presupuesto": presupuesto}
        
        # Si hay múltiples días coincidentes, solicitar los índices al usuario
        selected_indices = self.prompt_user_for_indices(
            filtered_provision_datos, presupuesto, año, concepto, mes, sociedad
        )["Indices"]

        # Paso 4: Calcular la suma de provisión para los índices seleccionados por el usuario
        provision_sum = filtered_provision_datos.loc[selected_indices]["Importe"].sum()
        
        # Llamar a `get_diferencias` si la diferencia está fuera de la tolerancia
        if not self.check_tolerance(presupuesto, provision_sum, operation="subtract"):
            self.get_diferencias(
                expected_value=presupuesto,
                comparison_sum=provision_sum,
                año=año,
                concepto=concepto,
                mes=mes,
                sociedad=sociedad,
                indices=selected_indices
            )
        
        # Devolver los índices seleccionados y el presupuesto
        return {"Indices": selected_indices, "Presupuesto": presupuesto}

In [13]:
class ExpenditureCalculator(BaseCalculator):
    def __init__(self, helper, provision_calculator, tolerance=0.5):
        super().__init__(helper, tolerance)
        self.provision_calculator = provision_calculator

    def calculate_expenditure(self, concepto, año, mes, sociedad):
        # Paso 1: Filtrar los datos para el concepto, año, mes y sociedad especificados
        filtered_datos = self.helper.get_filtered_datos(concepto, año, mes, sociedad)
        filtered_expenditure_datos = self.helper.filter_by_account_type_1(filtered_datos)

        # Paso 2: Calcular el gasto total
        total_expenditure = filtered_expenditure_datos["Importe"].sum()

        # Paso 3: Manejar casos en los que el gasto total es mínimo o no hay datos de gasto
        if abs(total_expenditure) <= self.tolerance or filtered_expenditure_datos.empty:
            filtered_provision_datos = self.helper.filter_by_account_type_2(filtered_datos)
            if filtered_provision_datos.empty:
                return {"Indices": [], "Total Expenditure": total_expenditure}

        # Paso 4: Obtener la provisión desde la caché o calcularla si aún no está
        provision_result = self.provision_calculator.get_cached_provision(concepto, año, mes, sociedad)
        matched_indices = provision_result.get("Indices", [])

        # Paso 5: Filtrar los índices de provisión ya emparejados si existen
        filtered_provision_datos = self.helper.filter_by_account_type_2(filtered_datos)
        if matched_indices:
            filtered_provision_datos = filtered_provision_datos[~filtered_provision_datos.index.isin(matched_indices)]

        # **Check 1: Si no hay datos de provisión restantes y el gasto total es cero, retornar de inmediato**
        if filtered_provision_datos.empty and abs(total_expenditure) <= self.tolerance:
            return {"Indices": [], "Total Expenditure": 0}

        # **Check 2: Si no hay datos de provisión restantes y el gasto total es no nulo, calcular diferencia**
        if filtered_provision_datos.empty and abs(total_expenditure) > self.tolerance:
            self.get_diferencias(
                expected_value=total_expenditure,
                comparison_sum=0,
                año=año,
                concepto=concepto,
                mes=mes,
                sociedad=sociedad,
                indices=[]
            )
            return {"Indices": [], "Total Expenditure": total_expenditure}

        # Paso 6: Intentar emparejar el gasto con los datos de provisión
        match_count = 0
        match_index = []
        match_day = None
        match_centro = None
        for idx, row in filtered_provision_datos.iterrows():
            provision_importe = row["Importe"]
            if self.check_tolerance(total_expenditure, provision_importe, operation="add"):
                match_count += 1
                match_index.append(idx)
                match_day = row["Fe.contabilización"].day
                match_centro = row["Calculated_Centro_de_beneficio"]

        # Paso 7: Si se encuentra exactamente 1 coincidencia, realizar la verificación adicional
        if match_count == 1:
            matching_expenditure_rows = filtered_expenditure_datos[
                (filtered_expenditure_datos["Fe.contabilización"].dt.day == match_day) &
                (filtered_expenditure_datos["Calculated_Centro_de_beneficio"] == match_centro)
            ]

            if matching_expenditure_rows.empty:
                additional_provision_matches = filtered_provision_datos[
                    (filtered_provision_datos["Fe.contabilización"].dt.day == match_day) &
                    (filtered_provision_datos["Calculated_Centro_de_beneficio"] == match_centro) &
                    (~filtered_provision_datos.index.isin(match_index))
                ]

                if additional_provision_matches.empty:
                    return {"Indices": match_index, "Total Expenditure": total_expenditure}
                else:
                    additional_indices = additional_provision_matches.index.tolist()
                    all_indices = match_index + additional_indices
                    provision_sum = filtered_provision_datos.loc[all_indices, "Importe"].sum()

                    self.get_diferencias(
                        expected_value=total_expenditure,
                        comparison_sum=provision_sum,
                        año=año,
                        concepto=concepto,
                        mes=mes,
                        sociedad=sociedad,
                        indices=all_indices
                    )

                    return {"Indices": all_indices, "Total Expenditure": total_expenditure}

        # Paso 8: Intentar emparejamiento adicional si no se encontró una coincidencia exacta
        remaining_provision_datos = filtered_provision_datos
        provision_days = remaining_provision_datos["Fe.contabilización"].dt.day
        unique_day = provision_days.mode()[0] if provision_days.nunique() == 1 else None
        centro_beneficio = remaining_provision_datos["Calculated_Centro_de_beneficio"].unique()

        if unique_day is not None:
            remaining_expenditure_datos = filtered_datos[
                ~filtered_datos.index.isin(matched_indices) & 
                filtered_datos["Calculated_Centro_de_beneficio"].isin(centro_beneficio)
            ]

            expenditure_combined = remaining_expenditure_datos[
                remaining_expenditure_datos["Fe.contabilización"].dt.day == unique_day
            ]

            combined_expenditure_sum = expenditure_combined["Importe"].sum()
            type_1_expenditure_sum = expenditure_combined[
                expenditure_combined["Cuenta"].apply(self.helper.get_account_type) == 1
            ]["Importe"].sum()

            total_expenditure -= type_1_expenditure_sum

            if self.check_tolerance(total_expenditure, combined_expenditure_sum, operation="add"):
                return {
                    "Indices": expenditure_combined.index.tolist(),
                    "Total Expenditure": total_expenditure
                }
        
        if not expenditure_combined.empty:
            indices_to_use = expenditure_combined.index.tolist()
            self.get_diferencias(
                expected_value=total_expenditure,
                comparison_sum=combined_expenditure_sum,
                año=año,
                concepto=concepto,
                mes=mes,
                sociedad=sociedad,
                indices=indices_to_use
            )

        return {
            "Indices": indices_to_use,
            "Total Expenditure": total_expenditure
        }

In [15]:
helper = FilterHelper(datos,conc_250,data_loader.cuentas_map)
provision_calculator = ProvisionCalculator(helper,presupuesto_combined)
expenditure_calculator = ExpenditureCalculator(helper,provision_calculator)

In [17]:
import pandas as pd

class ReporteConciliacion:
    def __init__(self, helper, provision_calculator, datos):
        self.helper = helper
        self.provision_calculator = provision_calculator
        self.datos = datos
        self.expenditure_calculator = ExpenditureCalculator(helper, provision_calculator)
        self.gasto_real_y_presupuesto = []
        self.provision_indices = []
        self.cancelacion_indices = []

    def reporte_final(self):
        """
        Genera el reporte consolidado de gasto real y presupuesto para cada combinación única de Año, Concepto, Mes y Sociedad.
        Devuelve 4 DataFrames: gasto_real_y_presupuesto_df, provision_rows_df, cancelacion_rows_df y
        diferencias_provision_df.
        """
        # Reiniciar las diferencias en provision_calculator y expenditure_calculator
        self.provision_calculator.diferencias = {}
        self.expenditure_calculator.diferencias = {}

        # Reiniciar las listas acumulativas para cada ejecución
        self.gasto_real_y_presupuesto = []
        self.provision_indices = []
        self.cancelacion_indices = []

        # Extraer combinaciones únicas de Año, Concepto, Mes y Sociedad
        unique_combinations = self.datos[['Año', 'Concepto', 'Mes', 'Sociedad']].drop_duplicates()

        for _, row in unique_combinations.iterrows():
            año = row['Año']
            concepto = row['Concepto']
            mes = row['Mes']
            sociedad = row['Sociedad']

            print(f"\nProcesando: Año: {año}, Concepto: '{concepto}', Mes: {mes}, Sociedad: {sociedad}")

            # Paso 1: Calcular provisión y obtener índices
            provision_result = self.provision_calculator.get_cached_provision(concepto, año, mes, sociedad)
            current_provision_indices = provision_result.get("Indices", [])
            presupuesto = provision_result.get("Presupuesto", 0)

            # Agregar los índices de la provisión actual a la lista acumulativa
            self.provision_indices.extend(current_provision_indices)

            # Paso 2: Calcular gasto real y obtener índices de cancelación
            expenditure_result = self.expenditure_calculator.calculate_expenditure(concepto, año, mes, sociedad)
            current_expenditure_indices = expenditure_result.get("Indices", [])
            total_expenditure = expenditure_result.get("Total Expenditure", 0)

            # Almacenar los índices de cancelación
            self.cancelacion_indices.extend(current_expenditure_indices)

            # Crear resumen para esta combinación y almacenarlo
            resumen = {
                "Año": año,
                "Concepto": concepto,
                "Mes": mes,
                "Sociedad": sociedad,
                "Gasto Real": total_expenditure,
                "Presupuesto": presupuesto,
                "Provision Indices": current_provision_indices,
                "Expenditure Indices": current_expenditure_indices
            }
            self.gasto_real_y_presupuesto.append(resumen)

        # Convertir el resumen en un DataFrame
        gasto_real_y_presupuesto_df = pd.DataFrame(self.gasto_real_y_presupuesto)

        # Crear DataFrames de las filas correspondientes a las provisiones y cancelaciones
        provision_rows_df = self.datos[self.datos.index.isin(self.provision_indices)]
        cancelacion_rows_df = self.datos[self.datos.index.isin(self.cancelacion_indices)]

        # Crear DataFrames para diferencias de provisión y cancelación usando las diferencias en provision_calculator y expenditure_calculator
        diferencias_provision_df = pd.DataFrame([
            {
                "Año": key[0],
                "Concepto": key[1],
                "Mes": key[2],
                "Sociedad": key[3],
                "Diferencia": -diff,
                "Tipo": "Provision"
            }
            for key, diff in self.provision_calculator.diferencias.items()
        ])

        diferencias_cancelacion_df = pd.DataFrame([
            {
                "Año": key[0],
                "Concepto": key[1],
                "Mes": key[2],
                "Sociedad": key[3],
                "Diferencia": diff,
                "Tipo": "Cancelacion"
            }
            for key, diff in self.expenditure_calculator.diferencias.items()
        ])

        # Unir los DataFrames de diferencias
        diferencias_df = pd.concat([diferencias_provision_df, diferencias_cancelacion_df], ignore_index=True)

        return (
            gasto_real_y_presupuesto_df, 
            provision_rows_df, 
            cancelacion_rows_df,
            diferencias_df
        )

    def guardar_resumen_final(self, gasto_real_y_presupuesto_df, provision_rows_df, cancelacion_rows_df):
        """
        Guarda los DataFrames de gasto_real_y_presupuesto_df, provision_rows_df y cancelacion_rows_df en un archivo Excel
        llamado "Resumen Final.xlsx" con hojas específicas para cada DataFrame.
        """
        with pd.ExcelWriter("Resumen Final.xlsx") as writer:
            gasto_real_y_presupuesto_df.to_excel(writer, sheet_name="Gasto Real y Presupuesto", index=False)
            provision_rows_df.to_excel(writer, sheet_name="Provision filas", index=False)
            cancelacion_rows_df.to_excel(writer, sheet_name="Cancelacion filas", index=False)
        print("Resumen guardado en 'Resumen Final.xlsx'")

    def guardar_diferencias_final(self, diferencias_df):
        """
        Guarda las diferencias en un archivo Excel llamado "Diferencias Final.xlsx" con una sola hoja llamada "Diferencias".
        """
        with pd.ExcelWriter("Diferencias Final.xlsx") as writer:
            diferencias_df.to_excel(writer, sheet_name="Diferencias", index=False)
        print("Diferencias guardadas en 'Diferencias Final.xlsx'")

In [19]:
helper = FilterHelper(datos,conc_250,data_loader.cuentas_map)

In [21]:
provision_calculator = ProvisionCalculator(helper,presupuesto_combined)

In [23]:
expenditure_calculator = ExpenditureCalculator(helper,provision_calculator)

In [25]:
reporte = ReporteConciliacion(helper,provision_calculator,datos)

In [27]:
gasto_real_y_presupuesto_df,provision_rows_df,cancelacion_rows_df,diferencias_df = reporte.reporte_final()


Procesando: Año: 2024, Concepto: 'BOTIQUIN MEDICO', Mes: 7, Sociedad: 1
Obteniendo resultado de provisión en caché para: (2024, 'BOTIQUIN MEDICO', 7, 1)

Procesando: Año: 2024, Concepto: 'BOTIQUIN MEDICO', Mes: 8, Sociedad: 1
Obteniendo resultado de provisión en caché para: (2024, 'BOTIQUIN MEDICO', 8, 1)

Procesando: Año: 2024, Concepto: 'BOTIQUIN MEDICO', Mes: 3, Sociedad: 1
Obteniendo resultado de provisión en caché para: (2024, 'BOTIQUIN MEDICO', 3, 1)

Procesando: Año: 2024, Concepto: 'BOTIQUIN MEDICO', Mes: 9, Sociedad: 1
Obteniendo resultado de provisión en caché para: (2024, 'BOTIQUIN MEDICO', 9, 1)

Procesando: Año: 2024, Concepto: 'BOTIQUIN MEDICO', Mes: 10, Sociedad: 1
Obteniendo resultado de provisión en caché para: (2024, 'BOTIQUIN MEDICO', 10, 1)

Procesando: Año: 2024, Concepto: 'BOTIQUIN MEDICO', Mes: 2, Sociedad: 1102
Obteniendo resultado de provisión en caché para: (2024, 'BOTIQUIN MEDICO', 2, 1102)

Procesando: Año: 2024, Concepto: 'BOTIQUIN MEDICO', Mes: 2, Socieda

,Cuenta,Año,Mes,Concepto,Fe.contabilización,Fecha de documento,Clase,Nº documento,Importe,Moneda,Clave contabiliz,Centro de beneficio,Sociedad,Referencia a factura,Cta.contrapartida,Texto,Usuario,Año Documento,Calculated_Centro_de_beneficio,Index
185407,5153060206,2024,2,POLIZA DE GMM,2024-02-29,2024-02-29,AB,4200000020,"$-44,921.71",MXN,50,5204012305,5204,4200000020,2503905400,CANCELACION GASTO,A8049997,2024,12305,185407
193184,5153060206,2024,2,POLIZA DE GMM,2024-02-28,2024-02-28,AB,4200000016,"$45,051.00",MXN,40,5204012305,5204,4200000016,2503905400,PROVISION,A8049997,2024,12305,193184


Porfavor, agregar los indices de la Provision separado por commas :  193184


Obteniendo resultado de provisión en caché para: (2024, 'POLIZA DE GMM', 2, 5204)

Procesando: Año: 2024, Concepto: 'POLIZA DE GMM', Mes: 3, Sociedad: 5204
Obteniendo resultado de provisión en caché para: (2024, 'POLIZA DE GMM', 3, 5204)

Procesando: Año: 2024, Concepto: 'POLIZA DE GMM', Mes: 6, Sociedad: 5204
Obteniendo resultado de provisión en caché para: (2024, 'POLIZA DE GMM', 6, 5204)

Procesando: Año: 2024, Concepto: 'POLIZA DE GMM', Mes: 7, Sociedad: 5204
Obteniendo resultado de provisión en caché para: (2024, 'POLIZA DE GMM', 7, 5204)

Procesando: Año: 2024, Concepto: 'POLIZA DE GMM', Mes: 8, Sociedad: 5202
Obteniendo resultado de provisión en caché para: (2024, 'POLIZA DE GMM', 8, 5202)

Procesando: Año: 2024, Concepto: 'POLIZA DE GMM', Mes: 8, Sociedad: 5204
Obteniendo resultado de provisión en caché para: (2024, 'POLIZA DE GMM', 8, 5204)

Procesando: Año: 2024, Concepto: 'POLIZA DE GMM', Mes: 10, Sociedad: 5505
Obteniendo resultado de provisión en caché para: (2024, 'POLIZA

,Cuenta,Año,Mes,Concepto,Fe.contabilización,Fecha de documento,Clase,Nº documento,Importe,Moneda,Clave contabiliz,Centro de beneficio,Sociedad,Referencia a factura,Cta.contrapartida,Texto,Usuario,Año Documento,Calculated_Centro_de_beneficio,Index
181437,5153060206,2024,5,POLIZA DE GMM,2024-05-31,2024-05-31,AB,4200000195,"$-650,000.00",MXN,50,1106012305,1106,4200000195,2503901501,CANCEL.GSTO,A8049997,2024,12305,181437
187329,5153060206,2024,5,POLIZA DE GMM,2024-05-30,2024-05-30,AB,4200000191,"$913,386.00",MXN,40,1106012305,1106,4200000191,2503905500,PROVISION,A8049997,2024,12305,187329


...

Porfavor, agregar los indices de la Provision separado por commas :  202108,204465,207073,207241,207630


Obteniendo resultado de provisión en caché para: (2024, 'SALUD Y BIENESTAR', 2, 1102)

Procesando: Año: 2024, Concepto: 'SALUD Y BIENESTAR', Mes: 5, Sociedad: 5202
Obteniendo resultado de provisión en caché para: (2024, 'SALUD Y BIENESTAR', 5, 5202)

Procesando: Año: 2024, Concepto: 'SALUD Y BIENESTAR', Mes: 4, Sociedad: 5202
Obteniendo resultado de provisión en caché para: (2024, 'SALUD Y BIENESTAR', 4, 5202)

Procesando: Año: 2024, Concepto: 'SALUD Y BIENESTAR', Mes: 9, Sociedad: 1106
Obteniendo resultado de provisión en caché para: (2024, 'SALUD Y BIENESTAR', 9, 1106)

Procesando: Año: 2024, Concepto: 'SALUD Y BIENESTAR', Mes: 10, Sociedad: 5505
Obteniendo resultado de provisión en caché para: (2024, 'SALUD Y BIENESTAR', 10, 5505)

Procesando: Año: 2024, Concepto: 'SALUD Y BIENESTAR', Mes: 4, Sociedad: 1102
Obteniendo resultado de provisión en caché para: (2024, 'SALUD Y BIENESTAR', 4, 1102)

Procesando: Año: 2024, Concepto: 'SALUD Y BIENESTAR', Mes: 7, Sociedad: 1106
Obteniendo res

,Cuenta,Año,Mes,Concepto,Fe.contabilización,Fecha de documento,Clase,Nº documento,Importe,Moneda,Clave contabiliz,Centro de beneficio,Sociedad,Referencia a factura,Cta.contrapartida,Texto,Usuario,Año Documento,Calculated_Centro_de_beneficio,Index
205885,5153060208,2024,1,SALUD Y BIENESTAR,2024-01-31,2024-01-31,AB,4200000025,"$-1,300.80",MXN,50,1102012305,1102,4200000025,2503900200,CANCELACION GASTO,A8049997,2024,12305,205885
207628,5153060208,2024,1,SALUD Y BIENESTAR,2024-01-30,2024-01-30,AB,4200000019,"$19,775.00",MXN,40,1102012305,1102,4200000019,2503905400,PROVISION,A8049997,2024,12305,207628


Porfavor, agregar los indices de la Provision separado por commas :  207628


Obteniendo resultado de provisión en caché para: (2024, 'SALUD Y BIENESTAR', 1, 1102)

Procesando: Año: 2024, Concepto: 'SALUD Y BIENESTAR', Mes: 10, Sociedad: 5202
Obteniendo resultado de provisión en caché para: (2024, 'SALUD Y BIENESTAR', 10, 5202)

Procesando: Año: 2024, Concepto: 'SALUD Y BIENESTAR', Mes: 9, Sociedad: 1105
Obteniendo resultado de provisión en caché para: (2024, 'SALUD Y BIENESTAR', 9, 1105)

Procesando: Año: 2024, Concepto: 'SALUD Y BIENESTAR', Mes: 1, Sociedad: 1105
Obteniendo resultado de provisión en caché para: (2024, 'SALUD Y BIENESTAR', 1, 1105)

Procesando: Año: 2024, Concepto: 'SALUD Y BIENESTAR', Mes: 1, Sociedad: 5202
Obteniendo resultado de provisión en caché para: (2024, 'SALUD Y BIENESTAR', 1, 5202)

Procesando: Año: 2024, Concepto: 'SALUD Y BIENESTAR', Mes: 1, Sociedad: 1106
Obteniendo resultado de provisión en caché para: (2024, 'SALUD Y BIENESTAR', 1, 1106)

Procesando: Año: 2024, Concepto: 'SALUD Y BIENESTAR', Mes: 3, Sociedad: 1106
Obteniendo res

In [29]:
gasto_real_y_presupuesto_df

,Año,Concepto,Mes,Sociedad,Gasto Real,Presupuesto,Provision Indices,Expenditure Indices
0,2024,BOTIQUIN MEDICO,7,1,"$484,226.87","$95,397.00",[424],"[677, 2566]"
1,2024,BOTIQUIN MEDICO,8,1,"$67,740.00","$95,397.00",[1452],[2102]
2,2024,BOTIQUIN MEDICO,3,1,"$257,250.22","$95,397.00",[1067],"[244, 2989]"
3,2024,BOTIQUIN MEDICO,9,1,"$79,293.82","$95,397.00",[239],[1757]
4,2024,BOTIQUIN MEDICO,10,1,"$82,395.04","$95,397.00",[2776],[2193]
...,...,...,...,...,...,...,...,...
895,2024,SERVICIOS DE APOYO,6,1105,$0.00,$830.00,[211997],[]
896,2024,SERVICIOS DE APOYO,10,1105,$0.00,$830.00,[212700],[]
897,2024,SERVICIOS DE APOYO,10,5505,$0.00,$244.00,[212737],[]
898,2024,SERVICIOS DE APOYO,4,1105,$0.00,$830.00,[212843],[]


In [33]:
provision_rows_df

,Cuenta,Año,Mes,Concepto,Fe.contabilización,Fecha de documento,Clase,Nº documento,Importe,Moneda,Clave contabiliz,Centro de beneficio,Sociedad,Referencia a factura,Cta.contrapartida,Texto,Usuario,Año Documento,Calculated_Centro_de_beneficio,Index
6,5153060301,2024,2,BOTIQUIN MEDICO,2024-02-28,2024-02-28,AB,4200000059,"$-1,046.00",MXN,50,1102012305,1102,4200000059,5153060103,PROVISION,A8049997,2024,12305,6
43,5153060301,2024,1,BOTIQUIN MEDICO,2024-01-30,2024-01-30,AB,4200000010,$108.00,MXN,40,5505012305,5505,4200000010,2503902510,PROVISION,A8049997,2024,12305,43
182,5153060301,2024,10,BOTIQUIN MEDICO,2024-10-30,2024-10-30,AB,4200000312,$293.00,MXN,40,1106012305,1106,4200000312,2503900200,PROVISION,A3604896,2024,12305,182
192,5153060301,2024,9,BOTIQUIN MEDICO,2024-09-27,2024-09-27,AB,4200000195,$108.00,MXN,40,1105012305,1105,4200000195,2503900200,PROVISION,A8049997,2024,12305,192
239,5153060301,2024,9,BOTIQUIN MEDICO,2024-09-27,2024-09-27,AB,4200026471,"$95,397.00",MXN,40,12305,1,4200026471,2503904300,PROVISION,A8028185,2024,12305,239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212786,5153060205,2024,7,SERVICIOS DE APOYO,2024-07-30,2024-07-30,AB,4200020350,"$1,739,966.00",MXN,40,12305,1,4200020350,2503904300,PROVISION,A8028185,2024,12305,212786
212843,5153060205,2024,4,SERVICIOS DE APOYO,2024-04-29,2024-04-29,AB,4200000084,$830.00,MXN,40,1105012305,1105,4200000084,2503900200,PROVISION,A3604896,2024,12305,212843
212860,5153060205,2024,3,SERVICIOS DE APOYO,2024-03-26,2024-03-26,AB,4200000096,"$56,621.00",MXN,40,1102012305,1102,4200000096,2503905500,PROVISION,A8049997,2024,12305,212860
212870,5153060205,2024,9,SERVICIOS DE APOYO,2024-09-27,2024-09-27,AB,4200000290,$766.00,MXN,40,1106012305,1106,4200000290,2503900200,PROVISION,A8049997,2024,12305,212870


In [31]:
cancelacion_rows_df

,Cuenta,Año,Mes,Concepto,Fe.contabilización,Fecha de documento,Clase,Nº documento,Importe,Moneda,Clave contabiliz,Centro de beneficio,Sociedad,Referencia a factura,Cta.contrapartida,Texto,Usuario,Año Documento,Calculated_Centro_de_beneficio,Index
244,5153060301,2024,3,BOTIQUIN MEDICO,2024-03-27,2024-03-27,AB,4200008594,"$-226,930.00",MXN,50,12305,1,4200008594,2503904300,CANCELACION GASTO,A8028185,2024,12305,244
655,5106030100,2024,4,BOTIQUIN MEDICO,2024-04-30,2024-04-30,AB,4200011555,"$-31,941.33",MXN,50,12305,1,4200011555,2503904300,CANCELACION GASTO,A8028185,2024,12305,655
677,5106030100,2024,7,BOTIQUIN MEDICO,2024-07-31,2024-07-31,AB,4200020371,"$-362,656.76",MXN,50,12305,1,4200020371,2503904300,CANCELACION GASTO,A8028185,2024,12305,677
722,5153060301,2024,5,BOTIQUIN MEDICO,2024-05-31,2024-05-31,AB,4200014519,"$-95,397.00",MXN,50,12305,1,4200014519,2503904300,CANCELACION GASTO,A8028185,2024,12305,722
1003,5153060301,2024,10,BOTIQUIN MEDICO,2024-10-31,2024-10-31,AB,4200000368,$-160.90,MXN,50,1102012305,1102,4200000368,2503901501,CANCELACION GASTO,A3604896,2024,12305,1003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212573,5106020105,2024,4,SERVICIOS DE APOYO,2024-04-30,2024-04-30,AB,4200000171,"$-1,915.52",MXN,50,1106012305,1106,4200000171,2503905500,CANCELACION GASTO,A8049997,2024,12305,212573
212651,5153060205,2024,5,SERVICIOS DE APOYO,2024-05-31,2024-05-31,AB,4200014519,"$-1,311,331.62",MXN,50,12305,1,4200014519,2503904300,CANCELACION GASTO,A8028185,2024,12305,212651
212732,5153060205,2024,2,SERVICIOS DE APOYO,2024-02-29,2024-02-29,AB,4200000068,"$-89,527.28",MXN,50,1102012305,1102,4200000068,2503901501,CANCELACION GASTO,A8049997,2024,12305,212732
212767,5106020105,2024,7,SERVICIOS DE APOYO,2024-07-31,2024-07-31,BM,6297,"$-128,961.12",MXN,50,1102012305,1102,6297,2503901501,CANCELACION GASTO,A8049997,2024,12305,212767


In [35]:
diferencias_df

,Año,Concepto,Mes,Sociedad,Diferencia,Tipo
0,2024,POLIZA DE GMM,2,5204,"$-45,000.00",Provision
1,2024,POLIZA DE GMM,5,1106,"$650,000.00",Provision
2,2024,POLIZA DE GMM,5,1102,"$3,000,000.00",Provision
3,2024,POLIZA DE GMM,5,5202,"$700,000.00",Provision
4,2024,POLIZA DE GMM,5,5204,"$-92,000.00",Provision
5,2024,SALUD Y BIENESTAR,2,1102,"$-26,859.00",Provision
6,2024,SALUD Y BIENESTAR,1,1102,"$-26,859.00",Provision
7,2024,BOTIQUIN MEDICO,7,1,"$312,364.11",Cancelacion
8,2024,BOTIQUIN MEDICO,6,1,"$69,223.89",Cancelacion
9,2024,CUPON SALUD,6,1,"$-69,223.89",Cancelacion


In [39]:
reporte.guardar_resumen_final(gasto_real_y_presupuesto_df,provision_rows_df,cancelacion_rows_df)

Resumen guardado en 'Resumen Final.xlsx'


In [37]:
reporte.guardar_diferencias_final(diferencias_df)

Diferencias guardadas en 'Diferencias Final.xlsx'
